# 17-06 한국어 BERT의 다음 문장 예측(Next Sentence Prediction)

* 모든 BERT 실습은 Colab에서 진행한다고 가정합니다.

사전 학습된 한국어 BERT를 이용하여 다음 문장 예측을 실습해봅시다. 이번 실습을 위해서만이 아니라 앞으로 사전 학습된 BERT를 사용할 때는 transformers라는 패키지를 자주 사용하게 됩니다. 실습 환경에 transformers 패키지를 설치해둡시다.

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 9.2 MB/s eta 0:00:00


In [2]:
import transformers
transformers.__version__

'4.25.1'

# 1. 모델 로드

transformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다.

In [3]:
from transformers import TFBertForNextSentencePrediction

TFBertForNextSentencePrediction.from_pretrained('BERT 모델 이름')을 넣으면 두 개의 문장이 이어지는 문장 관계인지 여부를 판단하는 BERT 구조를 로드합니다.


In [4]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


# 2. 토크나이저 로드

In [5]:
from transformers import AutoTokenizer

In [6]:
import tensorflow as tf


AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [7]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 3. 테스트

이제 TFBertForNextSentencePrediction를 통해서 다음 문장을 예측해봅시다. 모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값인 logits을 리턴합니다. 해당 값을 소프트맥스 함수를 통과시킨 후 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도록 합니다.

인덱스 0 : 실제 다음 문장  
인덱스 1 : 서로 상관없는 문장

In [8]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [9]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


In [10]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')
print(encoding)

{'input_ids': <tf.Tensor: shape=(1, 33), dtype=int32, numpy=
array([[    2,  5791,  2440,  5339,  4713,  2104,  2124,  2259,  3708,
         2145,  3965,  6233,  4414,  2496,  2359,  2414,  3665, 31221,
         1751,  9900, 12190,    18,     3,  6239,  2116,  2112, 12752,
         3771,  2138,  4530,  2585, 10283,     3]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 33), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 33), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [11]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.4330907,  5.5216575]], dtype=float32)>

In [12]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
probs

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[3.5086850e-04, 9.9964917e-01]], dtype=float32)>

In [13]:
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]
